# Determine tracts with different income percentiles

### Load data

In [12]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
cwd = os.getcwd()

df_input = pd.read_csv('Output\INPUT_for_resilience_model_v2.csv').set_index('tract')
print list(df_input)

['county_id', 'county_name', 'pcwgt', 'n_hh', 'pcinc', 'pov_frac', 'pop_dep', 'hh_share', 'savings_per_hh', 'pension_ss_pc', 'hh_str_asset_value', 'mort_pc', 'peinc_AGR', 'peinc_MIN', 'peinc_UTI', 'peinc_CON', 'peinc_MAN', 'peinc_WHO', 'peinc_RET', 'peinc_TRA', 'peinc_INF', 'peinc_FIN', 'peinc_PRO', 'peinc_EDU', 'peinc_ART', 'peinc_OTH', 'peinc_GOV', 'hh_pub_ass_inc', 'n_hh_inc_less15', 'n_hh_inc_15_25', 'n_hh_inc_25_35', 'n_hh_inc_35_50', 'n_hh_inc_50_75', 'n_hh_inc_75_100', 'n_hh_inc_100_125', 'n_hh_inc_125_150', 'n_hh_inc_150_200', 'n_hh_inc_more200', 'hh_size', 'LIL_hh', 'LIL_pc', 'VLIL_hh', 'VLIL_pc', 'ELIL_hh', 'ELIL_pc', 'pov_lev_hh', 'pov_lev_pc', 'Emp_AGR', 'Emp_MIN', 'Emp_UTI', 'Emp_CON', 'Emp_MAN', 'Emp_WHO', 'Emp_RET', 'Emp_TRA', 'Emp_INF', 'Emp_FIN', 'Emp_PRO', 'Emp_EDU', 'Emp_ART', 'Emp_OTH', 'Emp_GOV', 'Unemp']


In [19]:
# define a function for weighted quantiles. input: x, q
# x: two-column data, the second column is weight. q: percentile
def wquantile(x,q):           
    xsort = x.sort_values(x.columns[0])
    xsort['index'] = range(len(x))
    p = q * x[x.columns[1]].sum()
    pop = float(xsort[xsort.columns[1]][xsort['index']==0])
    i = 0
    print 'I got in'
    while pop < p:
        pop = pop + float(xsort[xsort.columns[1]][xsort['index']==i+1])
        i = i + 1
    print 'I got out'
    return xsort[xsort.columns[0]][xsort['index']==i]


#Weight the precentiles
quantile_thresh = [0.15,0.5,0.85]
pcinc_quantiles = []#np.array(df_avg_H.pcinc.quantile(quantile_thresh))

for quantile in quantile_thresh:
    pcinc_quantiles.append(wquantile(df_input[['pcinc','pcwgt']],quantile).values[0])

pcinc_quantiles[len(quantile_thresh)-1] += 1
print df_input.pcwgt.loc[df_input.pcinc <= pcinc_quantiles[1]].sum()*1./df_input.pcwgt.sum()


I got in
I got out
I got in
I got out
I got in
I got out
0.500159049818


In [22]:
df_out = pd.DataFrame(0, index=df_input.index, columns=['poor_15','poor_50','rich_15','rich_50'])

df_out.poor_15[df_input.pcinc <= pcinc_quantiles[0]] = 1
df_out.poor_50[df_input.pcinc <= pcinc_quantiles[1]] = 1
df_out.rich_50[df_input.pcinc > pcinc_quantiles[1]] = 1
df_out.rich_15[df_input.pcinc >= pcinc_quantiles[2]] = 1

df_out.to_csv('./Output/insurance_penetration_flags.csv')